# **TPS - Oct 2021**

## **XGBoost & LightGBM & CatBoost Stacking**

### Thank you for visiting my notebook :)
### This notebook is for beginner like me **who wants to study stacking ensemble!**

#### **Stacking Ensemble** is a nice technique for forwarding you score.
#### As you can see below image, Stacking Ensemble needs some models for classification and meta-model for final prediction!

#### Here's what you need to do.
**Step1. Make your train, test data for training & prediction (Preprocessing)**

**Step2. Select some models for making stacking datasets!! (Train models and Making Datasets)**

**Step3. Select final model for meta-model!**

**Step4. With your meta-model, Train & Predict with stacking datasets ;)**

![Stacking Ensemble](http://rasbt.github.io/mlxtend/user_guide/classifier/StackingClassifier_files/stackingclassification_overview.png)

# **Import Library**

In [1]:
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# **Load Data**

In [2]:
train = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv',nrows=10**5)
test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')

train.head()

In [3]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [4]:
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

## **Checking Missing Values**

In [ ]:
print(train.isna().sum(), test.isna().sum())

# Feature Generation

In [5]:
y = train['target']
train.drop(columns = ['id', 'target'], inplace = True)
test.drop(columns = 'id', inplace = True)

In [15]:
B_feature1 = ['f22', 'f43']+list(train.columns[243:286])#f22, f43, f242 ~ f284
B_feature2 = list(train.columns[242:286])#f22, f43, f242 ~ f284
B_feature3 = ['f22', 'f43']
# train[B_feature2].head()

## **エルボー法**

### **正規化なし**

In [7]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
def elbow_method(X):
    wcss = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 10, n_init = 10, random_state = 10)
        kmeans.fit(X)
        wcss.append(kmeans.inertia_)
    
    plt.plot(range(1, 11), wcss)
    plt.title('The elbow method')
    plt.xlabel('Number of clusters')
    plt.ylabel('WCSS') 
    plt.show()
#     return wcss

In [8]:
elbow_method(train)

### **正規化あり**

#### **標準化**

In [16]:
df = train[B_feature2].copy()
train_scaler = (df - df.mean()) / df.std(ddof=0)

In [20]:
train_scaler.head()

In [21]:
train_scaler = pd.concat([train.drop(columns=B_feature2),train_scaler],axis=1)
train_scaler.head()

In [22]:
elbow_method(train_scaler)

#### **B_feature以外**

In [30]:
df = train.drop(columns=B_feature2).copy()

In [31]:
df.head()

In [32]:
elbow_method(df)

#### **B_featureのみ**

**正規化あり(標準化）**

In [33]:
df = train[B_feature2].copy()
train_scaler = (df - df.mean()) / df.std(ddof=0)

In [34]:
train_scaler.head()

In [35]:
elbow_method(train_scaler)

**標準化なし**

In [36]:
df = train[B_feature2].copy()
df.head()

In [37]:
elbow_method(df)

In [ ]:
# from sklearn.cluster import KMeans
# def make_f_clustering(num_cluster, feature, data):
# #     feature = df_importance[:column_num].index#<-これを使ってクラスタリングで新しい特徴量を作る
    
#     df = data[feature]

#     #num_cluster=10 # cluster数

#     clusters = KMeans(n_clusters = num_cluster, random_state = 0, n_jobs = -1)
#     clusters.fit(df)
#     centers = clusters.cluster_centers_
#     columns = df.columns
#     clust_features = pd.DataFrame(index = df.index)
#     for i in range(len(centers)):
#         clust_features[str(num_cluster)+'_clust_dist_'+ str(i + 1)] = (df[columns] - centers[i]).applymap(abs).apply(sum, axis = 1)
#     return clust_features
#     #clust_features.head()
    

In [ ]:
# df_T = make_f_clustering( 3, B_feature2, train)
# df_t = make_f_clustering(3, B_feature2, test)

In [ ]:
# train = pd.concat([train.drop(columns = B_feature2), df_T],axis=1)
# test = pd.concat([test.drop(columns = B_feature2), df_t],axis=1)

In [ ]:
train.head()

# **Modeling**

### **Stacking Data Loader**

In [ ]:
def Stacking_Data_Loader(model, model_name, train, y, test, fold):
    '''
    Put your train, test datasets and fold value!
    This function returns train, test datasets for stacking ensemble :)
    '''

    stk = StratifiedKFold(n_splits = fold, random_state = 42, shuffle = True)
    
    # Declaration Pred Datasets
    train_fold_pred = np.zeros((train.shape[0], 1))
    test_pred = np.zeros((test.shape[0], fold))
    
    for counter, (train_index, valid_index) in enumerate(stk.split(train, y)):
        x_train, y_train = train.iloc[train_index], y[train_index]
        x_valid, y_valid = train.iloc[valid_index], y[valid_index]

        print('------------ Fold', counter+1, 'Start! ------------')
        if model_name == 'cat':
            model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)])
        elif model_name == 'xgb':
            model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric = 'auc', verbose = 500, early_stopping_rounds = 200)
        else:
            model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric = 'auc', verbose = 500, early_stopping_rounds = 200)
            
        print('------------ Fold', counter+1, 'Done! ------------')
        
        train_fold_pred[valid_index, :] = model.predict_proba(x_valid)[:, 1].reshape(-1, 1)
        test_pred[:, counter] = model.predict_proba(test)[:, 1]
        
        del x_train, y_train, x_valid, y_valid
        gc.collect()
        
    test_pred_mean = np.mean(test_pred, axis = 1).reshape(-1, 1)
    
    del test_pred
    gc.collect()
    
    print('Done!')
    
    return train_fold_pred, test_pred_mean

### **Modeling**

#### Model's HyperParameters
* LGBM Param : https://www.kaggle.com/hiro5299834/tps-oct-2021-single-lightgbm
* Cat Param : https://www.kaggle.com/ranjeetshrivastav/tps-oct-21-catboost
* xgb Param : https://www.kaggle.com/rahulchauhan3j/tps-oct-2021-xgboost-pipeline-with-optuna#Model-Fit-and-Submission

Thanks for Sharing!

In [ ]:
lgb_params = {
    'objective': 'binary',
    'n_estimators': 20000,
    'random_state': 42,
    'learning_rate': 8e-3,
    'subsample': 0.6,
    'subsample_freq': 1,
    'colsample_bytree': 0.4,
    'reg_alpha': 10.0,
    'reg_lambda': 1e-1,
    'min_child_weight': 256,
    'min_child_samples': 20,
#     'device': 'cpu',
    'device': 'gpu',
}


xgb_params = {'n_estimators': 10000,
               'learning_rate': 0.03689407512484644,
               'max_depth': 8,
               'colsample_bytree': 0.3723914688159835,
               'subsample': 0.780714581166012,
               'eval_metric': 'auc',
               'use_label_encoder': False,
               'gamma': 0,
               'reg_lambda': 50.0,
#                'tree_method': 'hist',
                'tree_method': 'gpu_hist',
               'gpu_id': 0,
#                'predictor': 'cpu_predictor',
                'predictor': 'gpu_predictor',
               'random_state': 42}

cat_params = {'iterations': 17298,
               'learning_rate': 0.03429054860458741,
               'reg_lambda': 0.3242286463210283,
               'subsample': 0.9433911589913944,
               'random_strength': 22.4849972385133,
               'depth': 8,
               'min_data_in_leaf': 4,
               'leaf_estimation_iterations': 8,
               'task_type':"GPU",
               'bootstrap_type':'Poisson',
               'verbose' : 500,
               'early_stopping_rounds' : 200,
               'eval_metric' : 'AUC'}

In [ ]:
lgbm = LGBMClassifier(**lgb_params)

xgb = XGBClassifier(**xgb_params)

cat = CatBoostClassifier(**cat_params)

### **Stacking**

* Making train, test prediction array!
* Concat 3 arrays in 1 dataset

In [ ]:
cat_train, cat_test = Stacking_Data_Loader(cat, 'cat', train, y, test, 5)
del cat
gc.collect()

lgbm_train, lgbm_test = Stacking_Data_Loader(lgbm, 'lgbm', train, y, test, 5)
del lgbm
gc.collect()

xgb_train, xgb_test = Stacking_Data_Loader(xgb, 'xgb', train, y, test, 5)
del xgb
gc.collect()

### **Final Stacking Datasets!**

In [ ]:
stack_x_train = np.concatenate((cat_train, lgbm_train, xgb_train), axis = 1)
stack_x_test = np.concatenate((cat_test, lgbm_test, xgb_test), axis = 1)
# stack_x_train = np.concatenate((lgbm_train, xgb_train), axis = 1)
# stack_x_test = np.concatenate(( lgbm_test, xgb_test), axis = 1)
del cat_train, lgbm_train, xgb_train, cat_test, lgbm_test, xgb_test
gc.collect()

stack_x_train

In [ ]:
stack_x_train.shape

### **Save stack_x_train**

In [ ]:
cols = ['cat3_fc1','lgbm3_fc1', 'xgb3_fc1']
df_stack = pd.DataFrame(stack_x_train,columns = cols)
df_stack.to_csv('stacking_train_clx3_fc1.csv', index = 0)
df_stack.head()

In [ ]:
df_test = pd.DataFrame(stack_x_test,columns = cols)
df_test.to_csv('stacking_test_clx3_fc1.csv', index = 0)
df_test.head()

## Level 2 Training

In [ ]:
# stk = StratifiedKFold(n_splits = 5)

# test_pred_lo = 0
# fold = 1
# total_auc = 0

# for train_index, valid_index in stk.split(stack_x_train, y):
#     x_train, y_train = stack_x_train[train_index], y[train_index]
#     x_valid, y_valid = stack_x_train[valid_index], y[valid_index]
    
#     lr = LogisticRegression(n_jobs = -1, random_state = 42, C = 5, max_iter = 2000)
#     lr.fit(x_train, y_train)
    
#     valid_pred_lo = lr.predict_proba(x_valid)[:, 1]
#     test_pred_lo += lr.predict_proba(stack_x_test)[:, 1]
#     auc = roc_auc_score(y_valid, valid_pred_lo)
#     total_auc += auc / 5
#     print('Fold', fold, 'AUC :', auc)
#     fold += 1
    
# print('Total AUC score :', total_auc)

# **Submission!**

## Blending Level 2 results

In [ ]:
# sub = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')
# sub['target'] = test_pred_lo
# sub.to_csv('sub.csv', index = 0)
# sub.head()

# Done!


## If you think this notebook is helpful for you, Please do not forget upvote!